In [62]:
import os
import librosa
from librosa import display
import numpy as np
from matplotlib import pyplot as plt
import time
import random


import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import datetime
from matplotlib import pyplot as plt
from pretty_confusion_matrix import pp_matrix_from_data

# Listing all samples

In [63]:
DATASET_PATH = "samples"
TRAIN_VAL_SPLIT = 0.1

samples = os.listdir(DATASET_PATH)
random.shuffle(samples)

# EmotionsDict = {'N': 'Neutral', 'F':'Fear', 'H':'Happiness', 'S':'Sadness', 'W':'Surprise', 'A':'Angry'}
EmotionsDict = {'N': 0, 'F': 1, 'H': 2, 'S': 3, 'W': 4, 'A': 5}
emotions = [EmotionsDict[x[3]] for x in samples]

# Preprocessing data and creating features

In [64]:
sr = 44100

SIG_LENGTH = 200

mfccs = []

for s in samples:
    t1 = time.time()

    sig, sr = librosa.load(f'{DATASET_PATH}/{s}', sr=sr)
    sig, _ = librosa.effects.trim(sig) # trimming beginning and ending silence


    # MFCC config
    WinLen = int(0.040 * sr) # 40 milisecond
    WinHop = WinLen // 2
    
    sig_mfcc = librosa.feature.mfcc(y=librosa.power_to_db(sig), sr=sr, n_mfcc=12, fmax=sr//2)

    # cutting every longer sequence than SIG_LENGTH and padding any sequence shorter than SIG_LENGTH
    if sig_mfcc.shape[1] < SIG_LENGTH:
        sig_mfcc = np.pad(sig_mfcc, [(0, 0), (0, SIG_LENGTH - sig_mfcc.shape[1])], mode='symmetric')
    elif sig_mfcc.shape[1] > SIG_LENGTH:
        sig_mfcc = sig_mfcc[:, :SIG_LENGTH]
    
    sig_mfcc = sig_mfcc.transpose()
    mfccs.append(sig_mfcc)

In [65]:
mfccs = np.array(mfccs)
emotions = np.array(emotions)

ds_len = mfccs.shape[0]
split_point = int(ds_len*(1-TRAIN_VAL_SPLIT))

avg = mfccs.mean()
std = mfccs.std()

mfccs = (mfccs - avg) / std # normalization

# mfccs = np.expand_dims(mfccs, axis=-1)

train_x = mfccs[:split_point]
train_y = emotions[:split_point]

val_x = mfccs[split_point:]
val_y = emotions[split_point:]


# print(mfccs.mean(), mfccs.std())

In [66]:
BATCH_SIZE = 64
LAYERS_ACTIVATION = 'relu'
LOSS_FUNCTION = 'sparse_categorical_crossentropy'

EPOCHS = 50

def get_model():
    model = tf.keras.Sequential([
              tf.keras.layers.Conv1D(256, 11),
              tf.keras.layers.MaxPool1D(3),
              tf.keras.layers.Conv1D(128, 7),
              tf.keras.layers.MaxPool1D(3),
              tf.keras.layers.Conv1D(64, 7),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(64, activation=LAYERS_ACTIVATION),
              tf.keras.layers.Dense(6, activation=tf.keras.activations.softmax)               
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=LOSS_FUNCTION, metrics=['accuracy'])
    return model

model = get_model()
history = model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/50
21/21 [==============================] - 4s 130ms/step - loss: 1.5505 - accuracy: 0.2246 - val_loss: 1.4939 - val_accuracy: 0.2215
Epoch 2/50
21/21 [==============================] - 2s 116ms/step - loss: 1.4049 - accuracy: 0.2664 - val_loss: 1.3808 - val_accuracy: 0.2953
Epoch 3/50
21/21 [==============================] - 3s 122ms/step - loss: 1.3986 - accuracy: 0.2552 - val_loss: 1.4030 - val_accuracy: 0.2752
Epoch 4/50
21/21 [==============================] - 2s 108ms/step - loss: 1.3783 - accuracy: 0.2985 - val_loss: 1.3770 - val_accuracy: 0.3154
Epoch 5/50
21/21 [==============================] - 3s 126ms/step - loss: 1.3919 - accuracy: 0.2806 - val_loss: 1.3566 - val_accuracy: 0.2752
Epoch 6/50
21/21 [==============================] - 2s 118ms/step - loss: 1.3662 - accuracy: 0.3060 - val_loss: 1.3561 - val_accuracy: 0.3624
Epoch 7/50
21/21 [==============================] - 2s 111ms/step - loss: 1.3486 - accuracy: 0.3306 - val_loss: 1.3467 - val_accuracy: 0.3356
Epoch 